# Compute heavy fractions

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.stats import binom

In [2]:
mid_data_path = Path('01_mid_data')
heavy_fractions_path = Path('02_heavy_fractions')

In [3]:
def _index_levels_to_int(df: pd.DataFrame, levels: list, axis: int) -> None:
    for level in levels:
        df.rename(int, level=level, axis=axis, inplace=True)

In [4]:
def import_mids(file_name: Path) -> pd.DataFrame:
    mids = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1, 2, 3, 4])
    _index_levels_to_int(mids, [2, 3, 4], axis=0)
    _index_levels_to_int(mids, [1], axis=1)
    return mids.sort_index()

In [5]:
# move to simpleflux package
def binomial_mid(n: int, p_heavy = 0.0107) -> np.array:
    return np.array([binom.pmf(k, n, p_heavy) for k in range(n+1)])

## Import MID data
We use only cell extract without standards, and only well-measured metabolites

### U-13C-methionine

In [6]:
met_mids = import_mids(mid_data_path / 'U13C-met_mids.csv')
met_mids.head(5)

metabolite                                                hcys     cyst       \
mi                                                           0   4    0    4   
matrix cell_type time_minutes with_standards replicate_nr                      
Cells  BJ-RAS    5            0              1             NaN NaN  1.0  0.0   
                                             2             NaN NaN  1.0  0.0   
                 15           0              1             NaN NaN  1.0  0.0   
                                             2             NaN NaN  1.0  0.0   
                 30           0              1             NaN NaN  1.0  0.0   

metabolite                                                 cys       met  \
mi                                                           0         0   
matrix cell_type time_minutes with_standards replicate_nr                  
Cells  BJ-RAS    5            0              1             1.0  0.049983   
                                             2             1.0  0.039748   
                 15           0              1             1.0  0.025675   
                                             2             1.0  0.028325   
                 30           0              1             1.0  0.027776   

metabolite                                                                     \
mi                                                                4         5   
matrix cell_type time_minutes with_standards replicate_nr                       
Cells  BJ-RAS    5            0              1             0.038196  0.911821   
                                             2             0.037279  0.922972   
                 15           0              1             0.039879  0.934446   
                                             2             0.042716  0.928959   
                 30           0              1             0.043632  0.928592   

metabolite                                                      sam            \
mi                                                                0         4   
matrix cell_type time_minutes with_standards replicate_nr                       
Cells  BJ-RAS    5            0              1             0.869159  0.000000   
                                             2             0.843118  0.000000   
                 15           0              1             0.510197  0.019301   
                                             2             0.504703  0.021287   
                 30           0              1             0.252247  0.033658   

metabolite                                                           sah       
mi                                                                5    0    4  
matrix cell_type time_minutes with_standards replicate_nr                      
Cells  BJ-RAS    5            0              1             0.130841  1.0  0.0  
                                             2             0.156882  1.0  0.0  
                 15           0              1             0.470503  0.0  1.0  
                                             2             0.474010  0.0  1.0  
                 30           0              1             0.714094  0.0  1.0

### D4-homocysteine

In [7]:
hcys_mids = import_mids(mid_data_path / 'D4-hcys_mids.csv')
hcys_mids.head(5)

metabolite                                                     hcys            \
mi                                                                0         4   
matrix cell_type time_minutes with_standards replicate_nr                       
Cells  BJ-RAS    5            0              1             0.025623  0.974377   
                                             2             0.022464  0.977536   
                 15           0              1             0.022170  0.977830   
                                             2             0.015744  0.984256   
                 30           0              1             0.026149  0.973851   

metabolite                                                     cyst            \
mi                                                                0         4   
matrix cell_type time_minutes with_standards replicate_nr                       
Cells  BJ-RAS    5            0              1             1.000000  0.000000   
                                             2             1.000000  0.000000   
                 15           0              1             1.000000  0.000000   
                                             2             1.000000  0.000000   
                 30           0              1             0.966616  0.033384   

metabolite                                                 cys       met  \
mi                                                           0         0   
matrix cell_type time_minutes with_standards replicate_nr                  
Cells  BJ-RAS    5            0              1             1.0  0.834521   
                                             2             1.0  0.807320   
                 15           0              1             1.0  0.835220   
                                             2             1.0  0.812176   
                 30           0              1             1.0  0.862720   

metabolite                                                                sam  \
mi                                                                4         0   
matrix cell_type time_minutes with_standards replicate_nr                       
Cells  BJ-RAS    5            0              1             0.165479  0.938673   
                                             2             0.192680  0.939687   
                 15           0              1             0.164780  0.893253   
                                             2             0.187824  0.878791   
                 30           0              1             0.137280  0.800938   

metabolite                                                                sah  \
mi                                                                4         0   
matrix cell_type time_minutes with_standards replicate_nr                       
Cells  BJ-RAS    5            0              1             0.061327  0.906442   
                                             2             0.060313  0.898653   
                 15           0              1             0.106747  0.850411   
                                             2             0.121209  0.828471   
                 30           0              1             0.199062  0.769718   

metabolite                                                           
mi                                                                4  
matrix cell_type time_minutes with_standards replicate_nr            
Cells  BJ-RAS    5            0              1             0.093558  
                                             2             0.101347  
                 15           0              1             0.149589  
                                             2             0.171529  
                 30           0              1             0.230282

## Estimate tracer purity

The expected tracer MID is $Bin(n, p_t)$ where $p_t$ is the purity. For the $n$'th MI the expected isotopomer fraction is $x_n = p_t^n$, so a simple estimate the tracer purity is $p_t = (x_n)^{1/n}$ 

### U-13C-methionine

In [8]:
met_fresh_medium_mi_fractions = met_mids.loc[('Medium', 'none', 0, 0)]['met'].iloc[0]
met_fresh_medium_mi_fractions

mi
0    0.000996
4    0.037897
5    0.961107
Name: 1, dtype: float64

In [9]:
met_tracer_purity = np.power(met_fresh_medium_mi_fractions.loc[5], 1/5)
met_tracer_purity

0.9920975111953164

Check residuals

In [10]:
met_expected_mid = [
    binom.pmf(mi, 5, met_tracer_purity)
    for mi in met_fresh_medium_mi_fractions.index
]
met_fresh_medium_mi_fractions - met_expected_mid

mi
0    9.957899e-04
4   -3.811084e-04
5   -2.220446e-16
Name: 1, dtype: float64

### D4-homocysteine

In [11]:
hcys_fresh_medium_mi_fractions = hcys_mids.loc[('Medium', 'none', 0, 0)]['hcys'].iloc[0]
hcys_fresh_medium_mi_fractions

mi
0    0.0013
4    0.9987
Name: 1, dtype: float64

In [12]:
hcys_tracer_purity = np.power(hcys_fresh_medium_mi_fractions.loc[4], 1/4)
hcys_tracer_purity

0.9996748734645691

Check residuals

In [13]:
hcys_expected_mid = [
    binom.pmf(mi, 4, hcys_tracer_purity)
    for mi in hcys_fresh_medium_mi_fractions.index
]
hcys_fresh_medium_mi_fractions - hcys_expected_mid

mi
0    0.0013
4    0.0000
Name: 1, dtype: float64

## Mixture model

We have a mixture model
$$y = x_0 y^0 + x_1 y^1$$
where $y$ the observed MID, $y^0$ is the natural MID and $y^1$ is the tracer MID. Solve for coefficients $x$ such that $x_0 + x_1 = 1$. Since we do not measure all mass isotopomers of $y$, we can only determine the measured components up to an unknown scale factor, $c$ so that we observe $y' = c y$. Multiplying the above with $c$,
$$y' = c y = c x_0 y^0 + c x_1 y^1$$
Fitting this model to data yields the coefficients $x' = c x$. To recover the fractions $x$, we normalize $x$ to sum to 1.

For larger metabolites that contain the tracer as a moiety (e.g. SAM) the distribution $y^1$ is a convolution between the tracer distribution and the natural distribution over the remaining carbons.

In [14]:
def convolution_matrix(x_mid: np.array, n_y: int) -> np.array:
        n_x = len(x_mid) - 1
        n_conv = n_x + n_y
        conv_mat = np.zeros(shape=[n_conv + 1, n_y + 1])
        for i in range(n_y + 1):
            conv_mat[i:(i + n_x + 1), i] = x_mid
        return conv_mat

In [15]:
def convolute(x_mid: np.array, y_mid: np.array) -> np.array:
    n_y = len(y_mid) - 1
    return convolution_matrix(x_mid, n_y) @ y_mid

In [16]:
def heavy_mid(n: int, n_tracer: int, tracer_purity: float) -> np.array:
    if n > n_tracer:
        return convolute(binomial_mid(n_tracer, tracer_purity), binomial_mid(n - n_tracer))
    else:
        return binomial_mid(n_tracer, tracer_purity)

In matrix form, the system to solve is
$$ (y^0 \ y^1) cx = cy $$
where $y, y^0, y^1$ contain the observed MIs only.

In [17]:
def estimate_heavy_fraction(mids: np.array, n_carbons: int, n_tracer_carbons: int, measured_mi: list, tracer_purity: float) -> np.array:
    if n_tracer_carbons == 0:
        return np.zeros(len(mids))
    mixture_basis = np.array([
        binomial_mid(n_carbons)[measured_mi],
        heavy_mid(n_carbons, n_tracer_carbons, tracer_purity)[measured_mi]
    ]).T
    rhs = mids.T
    coeff, _, _, _ = np.linalg.lstsq(mixture_basis, rhs, rcond=-1)
    return coeff[1] / np.clip(coeff.sum(axis=0), 1e-6, 1e+6)

In [18]:
def estimate_heavy_fraction_pd(mids: pd.DataFrame, n_carbons: int, n_tracer_carbons: int, tracer_purity: float) -> pd.Series:
    measured_mi = mids.columns.get_level_values('mi')
    return pd.Series(
        estimate_heavy_fraction(
            mids, n_carbons, n_tracer_carbons, measured_mi, tracer_purity),
        index=mids.index
    )

## Estimate heavy fractions

In [19]:
n_carbons = {'cys': 3, 'cyst': 7, 'hcys': 4, 'met': 5, 'sah': 14, 'sam': 15}
met_n_tracer_carbons = {'cys': 0, 'cyst': 4, 'hcys': 4, 'met': 5, 'sah': 4, 'sam': 5}
hcys_n_tracer_carbons = {'cys': 0, 'cyst': 4, 'hcys': 4, 'met': 4, 'sah': 4, 'sam': 4}

### U-13C-methioine

In [20]:
met_metabolites_to_use = ['cyst', 'met', 'sam']
met_mids_to_use = met_mids.loc['Cells'].xs(0, level='with_standards')[met_metabolites_to_use]
met_mids_to_use.head(5)

metabolite                          cyst            met                      \
mi                                     0    4         0         4         5   
cell_type time_minutes replicate_nr                                           
BJ-RAS    5            1             1.0  0.0  0.049983  0.038196  0.911821   
                       2             1.0  0.0  0.039748  0.037279  0.922972   
          15           1             1.0  0.0  0.025675  0.039879  0.934446   
                       2             1.0  0.0  0.028325  0.042716  0.928959   
          30           1             1.0  0.0  0.027776  0.043632  0.928592   

metabolite                                sam                      
mi                                          0         4         5  
cell_type time_minutes replicate_nr                                
BJ-RAS    5            1             0.869159  0.000000  0.130841  
                       2             0.843118  0.000000  0.156882  
          15           1             0.510197  0.019301  0.470503  
                       2             0.504703  0.021287  0.474010  
          30           1             0.252247  0.033658  0.714094

In [21]:
met_heavy_fractions = pd.DataFrame(
    {
        metabolite: estimate_heavy_fraction_pd(
            met_mids_to_use[metabolite],
            n_carbons=n_carbons[metabolite],
            n_tracer_carbons=met_n_tracer_carbons[metabolite],
            tracer_purity=met_tracer_purity
        )
        for metabolite in met_metabolites_to_use
    },
    index=met_mids_to_use.index
)
met_heavy_fractions.head(5)

cyst       met       sam
cell_type time_minutes replicate_nr                                  
BJ-RAS    5            1            -4.730827e-07  0.947336  0.128583
                       2            -4.730827e-07  0.958151  0.154254
          15           1            -4.730827e-07  0.972891  0.475179
                       2            -4.730827e-07  0.970010  0.479771
          30           1            -4.730827e-07  0.970564  0.735455

In [57]:
met_heavy_fractions.to_csv(heavy_fractions_path / '13C-met_heavy_fractions.csv')

### D4-homocysteine

In [23]:
hcys_metabolites_to_use = ['cyst', 'hcys', 'met', 'sah', 'sam']
hcys_mids_to_use = hcys_mids.loc['Cells'].xs(0, level='with_standards')[hcys_metabolites_to_use]
hcys_mids_to_use.head(5)

metabolite                               cyst                hcys            \
mi                                          0         4         0         4   
cell_type time_minutes replicate_nr                                           
BJ-RAS    5            1             1.000000  0.000000  0.025623  0.974377   
                       2             1.000000  0.000000  0.022464  0.977536   
          15           1             1.000000  0.000000  0.022170  0.977830   
                       2             1.000000  0.000000  0.015744  0.984256   
          30           1             0.966616  0.033384  0.026149  0.973851   

metabolite                                met                 sah            \
mi                                          0         4         0         4   
cell_type time_minutes replicate_nr                                           
BJ-RAS    5            1             0.834521  0.165479  0.906442  0.093558   
                       2             0.807320  0.192680  0.898653  0.101347   
          15           1             0.835220  0.164780  0.850411  0.149589   
                       2             0.812176  0.187824  0.828471  0.171529   
          30           1             0.862720  0.137280  0.769718  0.230282   

metabolite                                sam            
mi                                          0         4  
cell_type time_minutes replicate_nr                      
BJ-RAS    5            1             0.938673  0.061327  
                       2             0.939687  0.060313  
          15           1             0.893253  0.106747  
                       2             0.878791  0.121209  
          30           1             0.800938  0.199062

In [24]:
hcys_heavy_fractions = pd.DataFrame(
    {
        metabolite: estimate_heavy_fraction_pd(
            hcys_mids_to_use[metabolite],
            n_carbons=n_carbons[metabolite],
            n_tracer_carbons=hcys_n_tracer_carbons[metabolite],
            tracer_purity=hcys_tracer_purity
        )
        for metabolite in hcys_metabolites_to_use
    },
    index=hcys_mids_to_use.index
)
hcys_heavy_fractions

cyst      hcys       met  \
cell_type time_minutes replicate_nr                                     
BJ-RAS    5            1            -4.593566e-07  0.973315  0.159794   
                       2            -4.593566e-07  0.976601  0.186270   
          15           1            -4.593566e-07  0.976907  0.159115   
                       2            -4.593566e-07  0.983596  0.181539   
          30           1             3.206148e-02  0.972768  0.132411   
                       2             1.740887e-02  0.976848  0.156405   
          60           1             5.102135e-02  0.964485  0.230089   
                       2             5.314853e-02  0.968196  0.198892   
          300          1             4.291194e-01  0.963461  0.403578   
                       2             4.404467e-01  0.970300  0.501229   
          1440         1             8.303592e-01  1.000000  0.599460   
                       2             8.251840e-01  1.000000  0.586667   
BJ-TERT   5            1            -4.593566e-07  0.967441  0.094399   
                       2            -4.593566e-07  0.965242  0.164493   
          15           1            -4.593566e-07  0.966605  0.123148   
                       2            -4.593566e-07  1.000000  0.094606   
          30           1             1.425645e-02  0.975912  0.112947   
                       2             1.177419e-02  0.977433  0.140444   
          60           1             3.194046e-02  0.967212  0.115458   
                       2             3.125926e-02  0.961743  0.113577   
          300          1             3.413658e-01  1.000000  0.328670   
                       2             3.794063e-01  1.000000  0.274610   
          1440         1             8.983691e-01  1.000000  0.361051   
                       2             8.972190e-01  1.000000  0.359973   

                                          sah       sam  
cell_type time_minutes replicate_nr                      
BJ-RAS    5            1             0.090056  0.058944  
                       2             0.097586  0.057966  
          15           1             0.144331  0.102804  
                       2             0.165651  0.116804  
          30           1             0.222937  0.192457  
                       2             0.291220  0.213659  
          60           1             0.338568  0.320273  
                       2             0.386517  0.316930  
          300          1             0.558061  0.520881  
                       2             0.531719  0.533290  
          1440         1             0.718014  0.670520  
                       2             0.722595  0.641646  
BJ-TERT   5            1             0.096025  0.090070  
                       2             0.125807  0.088091  
          15           1             0.213681  0.182286  
                       2             0.229588  0.157434  
          30           1             0.345977  0.208646  
                       2             0.353136  0.199831  
          60           1             0.444748  0.251716  
                       2             0.479466  0.296056  
          300          1             0.629038  0.408706  
                       2             0.659595  0.415915  
          1440         1             0.642901  0.504933  
                       2             0.634632  0.519912

In [25]:
hcys_heavy_fractions.to_csv(heavy_fractions_path / 'D4-hcys_heavy_fractions.csv')